In [1]:
from multiprocessing import Pool
import pandas as pd
import numpy as np
import random
np.random.seed(1000)
random.seed(1000)

In [2]:
us_state_abbrev = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 
                   'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 
                   'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 
                   'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 
                   'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
                   'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 
                   'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 
                   'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
                   'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 
                   'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'}

county_cross = pd.read_csv("data/CountyCrossWalk_Zillow.csv")
county_data = pd.read_csv("data/County_time_series.csv", parse_dates=['Date'])
county_data = county_data[['Date', 'RegionName', 'ZHVIPerSqft_AllHomes']].dropna().reset_index(drop=True)

county_data.head(5)

,Date,RegionName,ZHVIPerSqft_AllHomes
0,1996-04-30,10001,58.0
1,1996-04-30,10003,76.0
2,1996-04-30,1003,62.0
3,1996-04-30,1009,44.0
4,1996-04-30,1017,49.0


In [ ]:
def plot_county(FIPS):
    import pandas as pd
    import numpy as np
    import random
    import plotly
    import plotly.offline as py
    import plotly.tools as tls
    import plotly.graph_objs as go
    import plotly.tools as tls
    import plotly.figure_factory as ff
    import numpy as np
    import random
    np.random.seed(1000)
    random.seed(1000)
    import keras
    keras.backend.clear_session()
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import LSTM
    from sklearn.metrics import mean_squared_error
    import os

    print(FIPS)
    statename = county_cross[county_cross['FIPS']== FIPS]['StateName'].values[0]
    d = county_data[county_data['RegionName'] == FIPS].reset_index(drop=True)
    ma = d['ZHVIPerSqft_AllHomes'].max()
    mi = d['ZHVIPerSqft_AllHomes'].min()
    d['ZHVIPerSqft_AllHomes'] = (d['ZHVIPerSqft_AllHomes']-mi)/(ma-mi)

    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    lookback = 2
    trainX, trainY = create_dataset(d['ZHVIPerSqft_AllHomes'].values, lookback)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    model = Sequential()
    model.add(LSTM(5, input_shape=trainX[0].shape))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=False)
    pred = model.predict(trainX).flatten()
    x = trainX[-1]
    y = []
    for i in range(24):
        train = np.array([x])
        pred = model.predict(train).flatten()
        y.append(pred[0]*(ma-mi) + mi)
        xt = list(x[0])
        xt = xt[1:] + [pred[0]]
        x = [xt]
    y = list(model.predict(trainX).flatten()*(ma-mi)+mi)+y
    previous = ['2017-01-31', '2017-02-28', '2017-03-30', '2017-04-30', '2017-05-31', '2017-06-30',
                '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31']
    new = ['2018-01-31 00:00:00', '2018-02-28 00:00:00', '2018-03-31 00:00:00', '2018-04-30 00:00:00', '2018-05-31 00:00:00', '2018-06-30 00:00:00', '2018-07-31 00:00:00', '2018-08-31 00:00:00', '2018-09-30 00:00:00', '2018-10-31 00:00:00', '2018-11-30 00:00:00', '2018-12-31 00:00:00',
           '2019-01-31 00:00:00', '2019-02-28 00:00:00', '2019-03-31 00:00:00', '2019-04-30 00:00:00', '2019-05-31 00:00:00', '2019-06-30 00:00:00', '2019-07-31 00:00:00', '2019-08-31 00:00:00', '2019-09-30 00:00:00', '2019-10-31 00:00:00', '2019-11-30 00:00:00', '2019-12-31 00:00:00']
    dates = previous[-lookback:] + new[:-lookback]

    d['ZHVIPerSqft_AllHomes'] = (d['ZHVIPerSqft_AllHomes']*(ma-mi))+mi
    true = d.iloc[:-lookback]
    pred = pd.DataFrame(columns=true.columns)
    pred['Date'] = list(d['Date'].values[:-lookback])+dates
    pred['ZHVIPerSqft_AllHomes'] = y
    pred['RegionName'] = FIPS
    dirs = os.listdir('UI/plots/states')
    a = us_state_abbrev[statename]
    if a not in dirs:
        os.mkdir('UI/plots/states/'+a)
    pred.to_csv('UI/plots/states/'+a+'/'+str(FIPS)+'.csv', index=False)
    return a

In [ ]:
pool = Pool(processes=36)
args = list(county_data['RegionName'].unique())
for i in pool.map(plot_county, args, chunksize=int(len(args)/36)+1):
    print(i)